In [1]:
import utils
import numpy as np
import glob 
import os
import pandas as pd
print(np.__version__)
from surface_parser_v2 import extract_and_save
import ray

1.21.0


Get Available Categories

In [2]:
def available_categories(ims_file_path: str, valid_surface: int):
    
    # load the imaris file
    data = utils.load_ims(ims_file_path)
    
    # get surface we want to parse
    surface_name = utils.get_object_names(full_data_file=data, 
                                           search_for='Surface')[valid_surface]
    
    # get all the statistics names
    surface_stats_names = utils.get_statistics_names(full_data_file=data, object_name=surface_name)
    
    # invert dictionary + name modifications
    # this step is a cosmetic step
    inverted_stats_names = utils.invert_stats_dict(surface_stats_names)
    inverted_stats_names = utils.flatten(inverted_stats_names)
        
    # reverse the stats names again such that key=num, value=name
    #final_stats_names = {v: k for k,v in inverted_stats_names.items()}
    
    np.savetxt('stats_categories_new.txt', list(inverted_stats_names.keys()), fmt='%s')
        
    print(f'Saved Stats Categories in directory: {os.path.dirname(os.path.abspath(ims_file_path))}')
    
    #return list(inverted_stats_names.keys())

In [3]:
def generate_available_categories(config_path: str):
    
    # load config path
    yaml = utils.load_yaml('config.yaml')

    # files to scan
    directories = yaml['data_dir']
    
    # valid surface
    valid_surface = int(yaml['valid_surface']) - 1
    
    for directory in directories:
        
        # grab all the files in the directory w/ .ims
        filenames = list(glob.glob(os.path.join(directory, '*.ims')))
                
        for filename in filenames: 
            
            # file path
            file_path = filename
            
            #print(file_path, filename, 'dafd')
            
            # get and save the available categories csv file
            available_categories(file_path, valid_surface)
            
            break
        
        break
    
    print('[info] Please Edit The Statistics File')

In [4]:
# RUN GENERATE CATEGORIES
#generate_available_categories('config.yaml')

Generate Statistics 

In [5]:
def generate_statistics(config_path: str):
    
    # load config path
    yaml = utils.load_yaml('config.yaml')

    # files to scan
    directories = yaml['data_dir']
    
    # get the stats categories
    stats_categories = utils.read_txt(yaml['stats_category_path'])
    
    # valid surface
    valid_surface = int(yaml['valid_surface']) - 1
    
    # create a list to hold ray subprocess
    processes = []
    
    for directory in directories:
        
        # grab all the files in the directory w/ .ims
        filenames = list(glob.glob(os.path.join(directory, '*.ims')))
        
        for filename in filenames:
                        
            # file path
            file_path = filename #os.path.join(directory, filename)
            
            # save_file_path
            save_path = os.path.splitext(filename)[0] + '.csv'
            #save_path = os.path.join(directory, save_path)
            
            # extract and save
            processes.append(extract_and_save.remote(file_path, valid_surface, stats_categories, save_path))
            
    print('info -- running subprocesses:')
    
    ray.get(processes) 

In [6]:
# RUN GENERATE STATISTICS
generate_statistics(config_path='config.yaml')

info -- running subprocesses:
(extract_and_save pid=70134) [info] working on file ../data/Surface_parser_column_swap_issue/Exp116-BACH1IKO42dpi_clec4f+tim4_10202022_New_[ii39_F480-AF532_FMO_3D_Merged_Image_40]_[ims1_2022-11-14T13-54-07.151].ims
(extract_and_save pid=70140) [info] working on file ../data/Surface_parser_column_swap_issue/Exp116-BACH1IKO42dpi_clec4f+tim4_10202022_New_[ii174_WTLi4_Merged_Image_175]_[ims1_2022-11-14T13-54-07.151].ims
(extract_and_save pid=70129) [info] working on file ../data/Surface_parser_column_swap_issue/Exp116-BACH1IKO42dpi_clec4f+tim4_10202022_New_[ii56_WTLi1_Merged_Image_57]_[ims1_2022-11-14T13-54-07.151].ims
(extract_and_save pid=70134) --- [warning] -- user requested category ID is NOT in current file
(extract_and_save pid=70134) [info] finished! 
(extract_and_save pid=70134) 
(extract_and_save pid=70140) --- [warning] -- user requested category ID is NOT in current file
(extract_and_save pid=70140) [info] finished! 
(extract_and_save pid=70140) 
(

In [7]:
# Working 

(extract_and_save pid=70129) [info] finished! 
(extract_and_save pid=70129) 
